Task1_0620. 네이버 영화 순위 사이트에서 영화제목, 가격, 타입(구매 or 대여) 정보를 가져와서 TITLE, PRICE, TYPE 3개의 컬럼과 100개의 데이터포인트로 구성된 데이터프레임을 출력하세요.

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import chardet

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

response=requests.get(url, headers=headers)
if response.status_code==200:
    encoding=chardet.detect(response.content)['encoding']
    soup = BeautifulSoup(response.text,'html.parser')  
else:
    print("HTTP request failed")

titles=soup.select('span.Title_title__s9o0D')
types=soup.select('span.Price_text__pRk_f')
prices=soup.select('span.Price_price__GqXqo')
title_li=[]; type_li=[]; price_li=[];
for i in range(len(titles)):
    title_li.append(titles[i].text)
    type_li.append(types[i].text)
    price_li.append(prices[i].text.replace('캐시',''))
df=pd.DataFrame({'Titles':title_li, 'Types':type_li, 'Prices':price_li})
df.head(100)

,Titles,Types,Prices
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150"
1,콩나물,구매,"1,000"
2,태극기 휘날리며 - 특별판,구매,"10,000"
3,설계자,대여,"11,000"
4,원피스 필름 레드,대여,"1,540"
...,...,...,...
81,마스크,대여,"1,300"
82,사운드 오브 뮤직,구매,"5,500"
83,마지막 한 걸음까지,구매,500
84,흑인 오르페,구매,500


In [6]:
#### Other Option using Copy/copy.selector

#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(2) > a > div.RankingNumber_ranking_area__p8B_q > em > span
## title
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > span
## Type
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_text__pRk_f
## Price
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_price__GqXqo



titles = []
types = []
prices = []
movie_tags = soup.select('div > ol > li')
for tag in movie_tags:
    title_tag = tag.select_one('a > div > span')
    if title_tag:
        titles.append(title_tag.text.strip())
    else:
        titles.append(None)
    type_tag = tag.select_one('a > div > div > div > div > div > span.Price_text__pRk_f')
    if type_tag:
        types.append(type_tag.text.strip())
    else:
        types.append(None)

    price_tag = tag.select_one('a > div > div > div > div > div > span.Price_price__GqXqo')
    if price_tag:
        prices.append(price_tag.text.strip().replace('캐시',''))
    else:
        prices.append(None)

df = pd.DataFrame({
    'Title': titles,
    'Type': types,
    'Price': prices
})
df

,Title,Type,Price
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150"
1,상승53,구매,"1,000"
2,태극기 휘날리며 - 특별판,구매,"10,000"
3,상승1,대여,"11,000"
4,상승11,대여,"1,540"
...,...,...,...
81,마스크,대여,"1,300"
82,사운드 오브 뮤직,구매,"5,500"
83,마지막 한 걸음까지,구매,500
84,흑인 오르페,구매,500


In [7]:
####################### 강사님
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 설정
url = 'https://serieson.naver.com/v3/movie/ranking/realtime'

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 영화 제목과 가격을 저장할 리스트
titles = []
prices = []
types = []

# 영화 제목과 가격 추출
movies = soup.select('.RankingList_ranking_list__N4QsH > li')

for movie in movies:
    title_tag = movie.select_one('.Title_title__s9o0D')
    if title_tag:
        title = title_tag.get_text(strip=True)
    
        # 가격 및 타입 추출
        price_items = movie.select('.Price_price_item__tOOqb')
        for price_item in price_items:
            type_tag = price_item.select_one('.Price_text__pRk_f')
            price_tag = price_item.select_one('.Price_price__GqXqo')
            if type_tag and price_tag:
                type_text = type_tag.get_text(strip=True)
                price = price_tag.get_text(strip=True)
                
                # 가격에서 "캐시" 및 SVG 태그 제거
                price = re.sub(r'캐시.*', '', price).strip()
                
                titles.append(title)
                prices.append(price)
                types.append(type_text)

# 데이터프레임 생성
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Type': types
})

# 데이터프레임 출력
df


,Title,Price,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150",구매
1,콩나물,"1,000",구매
2,태극기 휘날리며 - 특별판,"10,000",구매
3,설계자,"11,000",대여
4,원피스 필름 레드,"1,540",대여
...,...,...,...
81,마스크,"1,300",대여
82,사운드 오브 뮤직,"5,500",구매
83,마지막 한 걸음까지,500,구매
84,흑인 오르페,500,구매


Task2_0620. url = 'https://news.daum.net/politics#1'은 정치기사 1페이지인데 10페이지에 있는 기사를 모두 출력
"https://news.daum.net/politics#1"
(request랑 BS만 selenium 쓰지말고)

In [2]:
## 동적 컨텐츠 로딩: selenium 사용
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)

driver=webdriver.Chrome(options=options)

url = 'https://news.daum.net/politics#1'
driver.get(url)
time.sleep(3)

try:
    newsstand_link=driver.find_element(By.CSS_SELECTOR, '#timeline > ul > li:nth-child(1) > strong > a')
    print(newsstand_link.text)
except:
    print("뉴스 스탠드 링크를 찾을 수 없습니다.")

driver.quit()

“세계 반도체 人·水·電 싸움… 인재·용수·전력망 공급 정부가 나서야” [M 인터뷰]


In [15]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

articles_li=[]

for i in range(1,11, 1):
    url = f"https://news.daum.net/politics#{i}"
    response=requests.get(url, headers=headers)
    if response.status_code==200:
        encoding=chardet.detect(response.content)['encoding']
        soup = BeautifulSoup(response.text,'html.parser')  
    else:
        print("HTTP request failed")

    links=soup.select('a.link_txt')
    articles=[line.text.strip() for line in links]
    articles_li.extend(articles)


df=pd.DataFrame({"Articles":articles_li})
#df=pd.DataFrame({"Articles":links})
df.tail()
df.head()

,Articles
0,"국민의힘 유승민, 당대표 불출마 선언…'어대한'에 힘 실릴까?"
1,"원희룡, 출마 발표 전날 용산 찾아…尹대통령 ""존중한다"""
2,"[정치쇼] 김재섭 ""천하제일 '비윤' 호소대회…원희룡·尹 모종의 일 있었을 것"""
3,북러 고강도 군사 밀착에 한미도 면밀 대응…외교장관 긴급 통화
4,“영부인께 로비하고 싶습니다”… 권익위 ‘조롱’ 잇따라


Task3_0620. 앞에서 출력한 기사 리스트를 pandas 데이터프레임으로 변환 후 csv 파일로 저장 후 다시 불러오세요.
(컬럼은 인덱스 번호화 텍스트 2개 컬럼으로 구성된 DF)

In [19]:
import requests
import chardet
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

url = "https://news.daum.net/politics#1"

response=requests.get(url, headers=headers)
if response.status_code==200:
    encoding=chardet.detect(response.content)['encoding']
    soup = BeautifulSoup(response.text,'html.parser')  
else:
    print("HTTP request failed")

titles=soup.select('body > div > main > section > div > div > ul > li > strong > a')
indexes_li=[]; titles_li=[];
for i, tag in enumerate(titles, start=1):
    text=tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+',text)
    #print(f"{i}, {' '.join(matches)}")
    indexes_li.append(i); titles_li.append(' '.join(matches))

df_titles=pd.DataFrame({'indexes':indexes_li, 'titles':titles_li})
#df_titles.head()  
df_titles.to_csv('articles.csv', index=False)
df_reloaded=pd.read_csv('articles.csv',index_col=0)
df_reloaded.head()


,titles
indexes,
1,김현기 서울시의장 몽골 울란바토르에 소방차 10대 전달
2,북한 6 25 미제 반대 투쟁의 날 앞두고 원수 격멸 의지 굳건
3,박찬대 건희위원회로 전락한 권익위 고무줄 청탁금지법 지적
4,대통령실 야 단독소집 운영위 불참키로 야 증인 채택할듯
5,사실은 이렇습니다 과기정통부 출연연 지원 육성 관리 방안 6월 말 발표 예정


In [ ]:
################################# 강사님
mport re
import pandas as pd

soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')
results = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    result = ' '.join(matches)
    results.append({'Index':i, 'Text':result})
    
df = pd.DataFrame(results)

df.to_csv('news.csv', index=False)
df = pd.read_csv('news.csv')
df